# Tensor Network Quantum circuit

In [1]:
import numpy as np

We define states $|0\rangle$ and $|1\rangle$

In [2]:
s0 = np.array([1,0])
s1 = np.array([0,1])

And Pauli matrices X, Y, Z

In [3]:
opX = np.array([[0,1],[1,0]])
opZ = np.array([[1,0],[0,-1]])
opY = np.array([[0,-1j],[1j,0]])

In [4]:
def printState(s):
    st = np.ndarray.flatten(s)
    print(st[::-1],sep='')

In [5]:
printState(s0)
printState(s1)

[0 1]
[1 0]


## TN contractions with numpy.einsum

In [32]:
np.einsum('kj,ji',opX,opY)

array([[0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j]])

In [33]:
np.einsum('i,ij',s0,opX)

array([0, 1])

In [8]:
np.einsum('i,ij',s1,opZ)

array([ 0, -1])

In [9]:
np.einsum('i,ij,jk',s1,opZ,opZ)

array([0, 1])

## 2-qubit gates

In [10]:
cnot = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
cnot = np.reshape(cnot,[2,2,2,2])

In [11]:
res = np.einsum('i,j,ijkl',s1,s1,cnot)
print(res)

[[0 0]
 [1 0]]


In [12]:
printState(res)

[0 1 0 0]


### Test with 3 and 4 qubits

In [13]:
printState(np.einsum('i,j,k',s1,s1,s1))

[1 0 0 0 0 0 0 0]


In [14]:
res = np.einsum('i,j,ijkl',s1,s1,cnot)
res = np.einsum('ij,k,jksq',res,s1,cnot)
printState(res)

[0 1 0 0 0 0 0 0]


In [15]:
res = np.einsum('i,j,m,ijkl,lmsq',s1,s1,s1,cnot,cnot)

In [16]:
printState(res)

[0 1 0 0 0 0 0 0]


In [17]:
res = np.einsum('a,b,c,d,abij,jckl,ldmn',s1,s1,s1,s1,cnot,cnot,cnot)
printState(res)

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


### Einsum-path tests: contraction order

<img src="circuit.jpg" width=600 height=600 />

In [20]:
path = np.einsum_path('a,b,c,d,abij,jckl,ldmn',s1,s1,s1,s1,cnot,cnot,cnot,optimize='greedy')
print(path[1])
print(path[0])

  Complete contraction:  a,b,c,d,abij,jckl,ldmn->ikmn
         Naive scaling:  10
     Optimized scaling:  5
      Naive FLOP count:  7.168e+03
  Optimized FLOP count:  2.090e+02
   Theoretical speedup:  34.297
  Largest intermediate:  1.600e+01 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 abij,a->bij                b,c,d,jckl,ldmn,bij->ikmn
   4                 jckl,c->jkl                   b,d,ldmn,bij,jkl->ikmn
   4                 ldmn,d->lmn                      b,bij,jkl,lmn->ikmn
   3                   bij,b->ij                         jkl,lmn,ij->ikmn
   4                 ij,jkl->ikl                            lmn,ikl->ikmn
   5               ikl,lmn->ikmn                               ikmn->ikmn
['einsum_path', (0, 4), (1, 3), (1, 2), (0, 1), (0, 2), (0, 1)]


In [21]:
path = np.einsum_path('a,b,c,d,abij,jckl,ldmn',s1,s1,s1,s1,cnot,cnot,cnot,optimize='optimal')
print(path[1])
print(path[0])

  Complete contraction:  a,b,c,d,abij,jckl,ldmn->ikmn
         Naive scaling:  10
     Optimized scaling:  5
      Naive FLOP count:  7.168e+03
  Optimized FLOP count:  1.970e+02
   Theoretical speedup:  36.386
  Largest intermediate:  1.600e+01 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   2                     b,a->ab              c,d,abij,jckl,ldmn,ab->ikmn
   4                 jckl,c->jkl                 d,abij,ldmn,ab,jkl->ikmn
   4                 ldmn,d->lmn                    abij,ab,jkl,lmn->ikmn
   4                 ab,abij->ij                         jkl,lmn,ij->ikmn
   4                 ij,jkl->ikl                            lmn,ikl->ikmn
   5               ikl,lmn->ikmn                               ikmn->ikmn
['einsum_path', (0, 1), (0, 3), (0, 2), (0, 1), (0, 2), (0, 1)]
